# Module 2: Data wrangling using `pandas`

## Overview: Wrangle a messy geothermal dataset
Messy datasets are the norm in geoscience - this notebook will take you through python code very commonly used to import and wrangle some messy geothermal data. We will use `pandas`, a fast, powerful, flexible, and easy-to-use data analysis and manipulation tool.

For questions on this notebook, ask them on the [GEOL 557 slack](https://join.slack.com/t/minesgeo/shared_invite/zt-cqawm4lu-Zcfpf4mBLwjnksY6_umlKA)<a href="https://join.slack.com/t/minesgeo/shared_invite/zt-cqawm4lu-Zcfpf4mBLwjnksY6_umlKA">
<img src="https://cdn.brandfolder.io/5H442O3W/as/pl546j-7le8zk-ex8w65/Slack_RGB.svg" alt="Go to the GEOl 557 slack" width="100">
</a>

## Instructions
Work through this notebook - there will be several places where you need to fill-in-the-blank or write some code into an open cell. When you are finished, make sure to use the Colab menu (not the browser-level menu) to do the following:
- Expand all the sections - in the Colab menu, choose View --> Expand sections) 
- Save the notebook as a pdf, again using the Colab menu, using File --> Print --> Save as PDF. 

--- 
## Course
**GEOL 557 Earth Resource Data Science I: Fundamentals**. GEOL 557 forms part 2 of the four-part course series for the "Earth Resource Data Science" online graduate certificate at Mines - [learn more about the certificate here](https://online.mines.edu/er/)

Notebook created by **Zane Jobe** and **Thomas Martin**, [CoRE research group](https://core.mines.edu), Colorado School of Mines

[![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/ZaneJobe.svg?style=social&label=Follow%20%40ZaneJobe)](https://twitter.com/ZaneJobe)
and [![Twitter URL](https://img.shields.io/twitter/url/https/twitter.com/ThomasM_geo.svg?style=social&label=Follow%20%40ThomasM_geo)](https://twitter.com/ThomasM_geo) on Twitter 

# First, let's mount Google Drive and import `pandas`
We can import `pandas` normally because Google pre-loaded them into Colab. Google does not pre-load everything that you need, sometimes you might need to install it. If `pandas` wasn't a common library, you would have to install it using `pip` or some other method, but `pandas` is used everyday by millions of people. As an aside, the syntax to install a library would be `!pip install pandas`

---------------------

In [ ]:
import pandas as pd # this imports pandas to this notebook
import numpy as np

# these next two things shuoldnt need to be changed if you set up your Google Drive folder correctly (see Module 1)
folder_path = '../1_data/'
file_name = 'GeothermalPowerPlants_wikipedia.csv' # file name

## Let's load in a dataset
We load it with the pandas `read_csv` function into a [DataFrame](https://pandas.pydata.org/pandas-docs/stable/getting_started/intro_tutorials/01_table_oriented.html). We will name this DataFrame `df` and then preview it with the `head` function, giving us a visual view of the dataframe. Just like Excel, except `pandas` is so, so much better and you will see why soon. 

This table is from Wikipedia, from [here](https://en.wikipedia.org/wiki/List_of_geothermal_power_stations_in_the_United_States). I used https://wikitable2csv.ggor.de/ to make a csv from the table! Pretty nifty, but no guarantees on accuracy. 

In [ ]:

df= pd.read_csv(folder_path + file_name) # uses pandas to read in the csv as a 'DataFrame' called df
df.head()

By default, head looks at the first five rows using `df.head()`. Or you can specify how many rows to look at by putting a number in the parentheses (e.g., `df.head(2)`). You can also use `tail()` to look at the last five rows. 

You can also just list the column names using `df.columns` or `df.keys()`. We will use keys, as they behave similarly to keys in a python dict (i.e., dict = {key: value}). To only see the 3rd key, use `df.keys()[2]`

In [ ]:
df.keys()

OK, so we can see how the DataFrame is set up, and now we need to figure out what exactly is in each column. For this, we use `info()`, which gives a summary of the dataframe, and importantly, what their `dtypes` are:

In [ ]:
print(df.info())
np.shape(df)

So, there are 31 rows (you could also find this by using `len(df)`) and 8 columns `(len(df.keys())`. 

The Non-Null Count is how many values are not empty (or have Not-a-Number or NaN values). 

The Dtype is quite important for using and plotting the data. Notice that every column except for `Year` has a dtype of `object`, which isn't great, as object is the pandas-version of `string`. Year is listed as an int, which is good (i.e., Year shouldn't be a float or a string/object).

This probably means that there are weird values in some of the columns. This is pretty typical for datasets inputted manually (by someone typing in the numbers), outputted from an instrument, or OCRed. 

Let's take a look at one of the columns that's listed as an object dtype and should be a string, like the Name of the power plant. 

In [ ]:
df.Name

Now let's look at another column that's listed as object, but we know by looking at the df.head() that it should be a number (either `int` or `float`)

In [ ]:
df['Capacity(MW)'] # We have to use df['column name'] instead of df.column_name if the column has spaces or weird characters

Hmm, not good. One of the values has a comma (row 9), and two values  (rows 25, 30) have other weird text.

For this size of dataset, it may just be easier to do what we are going to do in Excel 😱. But, if this dataset was 5x-1000x larger, or you needed to do this to several hundred Excel files that were formatted the same way (which is typical when getting data from instruments or service companies), it is much easier to clean up using some python tools. Remember, ***automate the boring stuff!***

First, let's remove the commas from the dataframe.

Commonly, data you import from Excel will have random extra characters like commas - this happens when an Excel column is a number stored as text, so 1000 is 1,000, and python/pandas doesn't like that. Remember [this article?](https://www.forbes.com/sites/salesforce/2014/09/13/sorry-spreadsheet-errors/#32d3148856ab)

In [ ]:
df.replace(',','', regex=True, inplace=True) # searches the whole DataFrame and replaces commas with nothing
df['Capacity(MW)']

Progress! The comma went away. 

You can also use something like this if you have a really messy dataset (taken from [here](https://medium.com/analytics-vidhya/simplify-your-dataset-cleaning-with-pandas-75951b23568e)):

     spec_chars = ["!",'"',"#","%","&","'","(",")",
              "*","+",",","-",".","/",":",";","<",
              "=",">","?","@","[","\\","]","^","_",
              "`","{","|","}","~","–"]
     for char in spec_chars:
       df['column name'] = df['column name'].str.replace(char, '')

Be careful doing this type of thing, as it can really mess things up too, but it can save you a lifetime of manual Excel find-and-replace agony.

Now, it kind of sucks to have to type `Capacity(MW)', so let's change the names of two of the columns so they are easier to type, and drop the Ref column (we don't care about it for now).

In [ ]:
df = df.rename(columns={"Capacity(MW)": "Capacity_MW", "Annual Generation(GWh)": "AG_GWh"})
df = df.drop(columns=['Ref']) # gets rid of the column `Ref`
df.replace(',','', regex=True, inplace=True)

df.Capacity_MW

Ahh, much nicer. If you try to call that column by it's old name, you get a KeyError, which is saying that there isn't a key that's called "Capacity(MW)"

In [ ]:
# df['Capacity(MW)'] 

Now let's get rid of those other issues in the Capacity_MW column, where there is text after the numbers. We will use a Series function on the column, see details [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.html?highlight=series%20str#pandas.Series.str). Specifically, we will `split` on the space, and only keep the values before the space:

In [ ]:
df.Capacity_MW = df.Capacity_MW.str.split(" ", expand=True)[0] # splitting on the space delimiter and only keeping the first result [0]
df.Capacity_MW = pd.to_numeric(df.Capacity_MW) # Changing the type to numeric
print(df.Capacity_MW.dtype)
df.Capacity_MW

OK, now let's fix some other columns

In [ ]:
df.AG_GWh = df.AG_GWh.str.split(" ", expand=True)[0]
df.AG_GWh = pd.to_numeric(df.AG_GWh)
df.AG_GWh

For the Lat-Lon data, there are two ways to clean this column up(well, probably several, but we will only discuss two).

The first is to split it out to get only the decimal degrees part, and then separate the decimal degree into lat and lon values, creating new Series for each (and multiplying the lon by -1 to account for them all being West values):

In [ ]:
df.LocationDD = df.Location.str.split(r" / ", expand=True)[1]
df.LocationDD.head(2) # showing just the first two entries

#df.latDD_N  = df.LocationDD.str.split(r" *", expand=True)[0] 
#df.longDD_W = df.LocationDD.str.split(r" *", expand=True)[1]
df.latDD_N  = df.LocationDD.str.split(r" ", expand=True)[0] 
df.longDD_W = df.LocationDD.str.split(r" ", expand=True)[1]

df.latDD_N.replace('°N','', regex=True, inplace=True) # replacing the degree symbol with a space
df.longDD_W.replace('°W','', regex=True, inplace=True)

df.latDD_N.values



In [ ]:
df.latDD_N.replace('\ufeff','', regex=True, inplace=True) # doing more replacing
df.longDD_W.replace('\ufeff','', regex=True, inplace=True)

df['latDD_N'] = pd.to_numeric(df.latDD_N)
df['longDD_W'] = pd.to_numeric(df.longDD_W)

df.latDD_N = pd.to_numeric(df.latDD_N)
df.longDD_W = pd.to_numeric(df.longDD_W)

# Because it is in the west, easier to put a negative
df.longDD_W = df.longDD_W*-1 
df.longDD_W.values

The other is to use regex and applymap to each element of a Series

from https://realpython.com/python-data-cleaning-numpy-pandas/

DO THE APPLYMAP too!!!

r'^(\d{4})'
extr = df['Date of Publication'].str.extract(r'^(\d{4})', expand=False)
df['Date of Publication'] = pd.to_numeric(extr)

The regular expression above is meant to find any four digits at the beginning of a string, which suffices for our case. The above is a raw string (meaning that a backslash is no longer an escape character), which is standard practice with regular expressions.

The \d represents any digit, and {4} repeats this rule four times. The ^ character matches the start of a string, and the parentheses denote a capturing group, which signals to Pandas that we want to extract that part of the regex. (We want ^ to avoid cases where [ starts off the string.)

Let’s see what happens when we run this regex across our dataset:

https://realpython.com/regex-python/

This results in about one in every ten values being missing, which is a small price to pay for now being able to do computations on the remaining valid values:
df['Date of Publication'].isnull().sum() / len(df)


In [ ]:
df.groupby(['State']).size()

In [ ]:
df.groupby(['State']).sum()

![I'm done](https://media1.tenor.com/images/bfc7eb4615c496283ba14f079c2e499e/tenor.gif?itemid=10583001)